In [1]:
import numpy as np 
from astropy import units as u

import math
import numpy as np
import astropy.units as u
from astropy.constants import h, k_B, c

In [19]:
def compute_g_j(j):
    """g-factors from quantum numbers"""
    return (2 * j) + 1

def compute_wavelength(freq):
    """Wavelength (m) from the frequency (Hz)"""
    return (c / freq).to(u.cm)

def compute_E_j(j, B):
    """Energy of the energy of level (J)"""
    return j * (j + 1) * h * B

def _compute_Q_rot(j, E_j, T_ex):
    """Compute the partition function for a molecule given the excitation temperature and rotational constant."""
    return (2 * j + 1) * np.exp((-E_j / (k_B * T_ex)).decompose().value)

def compute_partition_function(T_ex, B, maxJ=10):
    """
    Compute the partition function for a molecule given the excitation
    temperature and rotational constant.
    """
    Q_rot = 0.0
    for j in range(maxJ):
        E_j = compute_E_j(j, B)
        Q_rot += _compute_Q_rot(j, E_j, T_ex)
    return Q_rot

def compute_J_T(T, freq):
    """equivalent Rayleigh-Jeans temperatures"""
    return (h * freq / k_B) / (np.exp((h * freq / (k_B * T)).decompose().value) - 1.0)

def compute_column_density(I, eI, T_ex, T_bg,
                           freq, A, B, q_l, q_u, maxJ=10):
    """
    Compute the column density and its error for a molecular transition,
    given its integrated intensity (in K km/s) and associated error.

    Parameters
    ----------
    I : astropy.units.Quantity
        Integrated intensity in K km/s.
    eI : astropy.units.Quantity
        Uncertainty in the integrated intensity (same units as I).
    T_ex : float
        Excitation temperature in K.
    T_bg : float
        Background temperature in K.
    freq : astropy.units.Quantity
        Transition frequency (e.g., in Hz).
    A : float
        Einstein A coefficient (s^-1).
    B : astropy.units.Quantity
        Rotational constant (in Hz).
    l : float
        Lower rotational quantum number.
    u : float
        Upper rotational quantum number.
    maxJ : int, optional
        Maximum quantum number used in the partition function sum (default=10).
        
    Returns
    -------
    N : astropy.units.Quantity
        Column density in cm^-2.
    eN : astropy.units.Quantity
        Uncertainty in the column density in cm^-2.
    """
    # -----------------------------
    # Fundamental constants and derived quantities
    # -----------------------------
    # g-factors from quantum numbers
    g_l = compute_g_j(q_l)
    g_u = compute_g_j(q_u)

    # Wavelength (m) from the frequency (Hz)
    wavelength = compute_wavelength(freq) # c in m/s, freq in Hz → wavelength in m
    print('Wavelength', wavelength)
    print('hv/k', (h * freq / k_B).to('K'))

    # Energy of the lower level (J)
    E_l = compute_E_j(q_l, B)
    print('E_l', E_l)

    # -----------------------------
    # Partition function: Q_rot = sum_{j=0}^{maxJ-1} (2j+1) exp[-E_j/(k_B T_ex)]
    # where E_j = j(j+1) h B.
    # -----------------------------
    Q_rot = compute_partition_function(T_ex, B, maxJ=maxJ)
    print('Q_rot', Q_rot)
    
    # -----------------------------
    # Brightness temperature correction:
    # J(T) = (h*nu/k_B) / (exp(h*nu/(k_B*T)) - 1)
    # -----------------------------
    # Note: we use .decompose().value to work with dimensionless arguments inside np.exp.
    J_ex = compute_J_T(T_ex, freq)
    J_bg = compute_J_T(T_bg, freq)
    print('J_ex', J_ex)
    print('J_bg', J_bg)
    
    # -----------------------------
    # Compute the prefactor (fcol) in the column density equation:
    #
    # N = fcol * (integrated intensity)    where:
    #
    #   fcol = (8π/(λ^3 A)) * (g_l/g_u) * 1/(J_ex - J_bg) *
    #          1/(1 - exp(-h*freq/(k_B*T_ex))) * Q_rot *
    #          1/(g_l * exp(-E_l/(k_B*T_ex)))
    # -----------------------------
    top_factor      = (8 * np.pi) / (wavelength**3 * A)
    factor_degen    = g_l / g_u
    factor_J        = 1.0 / (J_ex - J_bg)
    factor_exp_term = 1.0 / (1.0 - np.exp((-h * freq / (k_B * T_ex)).decompose().value))
    factor_level    = Q_rot / (g_l * np.exp((-E_l / (k_B * T_ex)).decompose().value))
    
    fcol = top_factor * factor_degen * factor_J * factor_exp_term * factor_level
    
    # -----------------------------
    # Compute the column density (N) in cm^-2.
    # (I is in K km/s; 'conv' converts to appropriate base units.)
    # -----------------------------
    N_val = I *  fcol
    N_err = eI * fcol

    # Likely the units come out as m^-2; convert to cm^-2.
    N = N_val.to(u.cm**-2)
    N_err = N_err.to(u.cm**-2)
    
    return N, N_err

In [25]:
# ------------------------------------------------------------------------------
# Example usage for N2H+ (1-0) and N2D+ (1-0)
# ------------------------------------------------------------------------------

# Example inputs (integrated intensities with errors in K km/s)
# Assume these values are measured from your observations.

# For N2H+ (1-0)
I_n2hp  = 1 * u.K * u.km / u.s
eI_n2hp = 0.1 * u.K * u.km / u.s
T_ex    = 4.5      # in K
T_bg    = 2.72548  # in K

# Constants for N2H+ (1-0)
freq_n2hp = 93.1762522e9 * u.Hz      # frequency in Hz
A_n2hp    = 3.6280e-5 / u.s                 # Einstein A [s^-1]
B_n2hp    = 46586.8713e6 * u.Hz       # Rotational constant [Hz]
l_n2hp    = 0.0                     # lower level quantum number
u_n2hp    = 1.0                     # upper level quantum number

N_n2hp, eN_n2hp = compute_column_density(I_n2hp, eI_n2hp,
                                            T_ex, T_bg,
                                            freq_n2hp, A_n2hp, B_n2hp,
                                            l_n2hp, u_n2hp, maxJ=10)

print("\n N2H+ (1-0):")
print("  Column density =", N_n2hp.to(u.dex(u.cm**-2)))
print("  Error           =", eN_n2hp.to(u.dex(u.cm**-2)))
print("\n")

# For N2D+ (1-0)
I_n2dp  = 1 * u.K * u.km / u.s
eI_n2dp = 0.05 * u.K * u.km / u.s

# We assume the same T_ex and T_bg.. 

# Use typical values for N2D+ (1-0) [example values]:
freq_n2dp = 77.109e9 * u.Hz         # frequency in Hz (approximate)
A_n2dp    = 2.07e-5 / u.s           # Einstein A [s^-1] (example scaling)
B_n2dp    = 38554.7479e6 * u.Hz     # Rotational constant [Hz]
l_n2dp    = 0.0                     # for a 1-0 transition, lower state = 0
u_n2dp    = 1.0                     # for 1-0, upper state = 1

N_n2dp, eN_n2dp = compute_column_density(I_n2dp, eI_n2dp,
                                            T_ex, T_bg,
                                            freq_n2dp, A_n2dp, B_n2dp,
                                            l_n2dp, u_n2dp, maxJ=10)
print("\nN2D+ (1-0):")
print("  Column density =", N_n2dp.to(u.dex(u.cm**-2)))
print("  Error           =", eN_n2dp.to(u.dex(u.cm**-2)))

Wavelength 0.32174771030337707 cm
hv/k 4.471754829730741 K
E_l 0.0 Hz J s
Q_rot 2.3827636147468145
J_ex 2.628475118143428 Hz K s
J_bg 1.0752272994394392 Hz K s

 N2H+ (1-0):
  Column density = 12.227539568307344 dex(1 / cm2)
  Error           = 11.227539568307344 dex(1 / cm2)


Wavelength 0.38879048878859795 cm
hv/k 3.7006483414419593 K
E_l 0.0 Hz J s
Q_rot 2.795146111564262
J_ex 2.9004699452960816 Hz K s
J_bg 1.2815569493857122 Hz K s

N2D+ (1-0):
  Column density = 12.326522066322145 dex(1 / cm2)
  Error           = 11.025492070658164 dex(1 / cm2)


In [26]:
# ------------------------------------------------------------------------------
# Example usage for C18O (1–0)
# ------------------------------------------------------------------------------

I_c18o  = 1.0 * u.K * u.km / u.s   # example integrated intensity
eI_c18o = 0.1 * u.K * u.km / u.s   # example uncertainty
T_ex    = 7.5                  # excitation temperature in K (typical for CO)
T_bg    = 2.72548                # background (CMB) temperature in K

# Constants for C18O (1–0)
freq_c18o = 109.7821734e9 * u.Hz   # frequency in Hz 
A_c18o    = 6.266e-8 / u.s         # Einstein A coefficient in s^-1 
B_c18o    = 54.89142e9 * u.Hz      # Rotational constant in Hz
l_c18o    = 0.0                  # lower level quantum number (J = 0)
u_c18o    = 1.0                  # upper level quantum number (J = 1)

N_c18o, eN_c18o = compute_column_density(I_c18o, eI_c18o,
                                         T_ex, T_bg,
                                         freq_c18o, A_c18o, B_c18o,
                                         l_c18o, u_c18o, maxJ=10)
print("\nC18O (1–0):")
print("  Column density =", N_c18o.to(u.cm**-2)/1e14)
print("  Error           =", eN_c18o.to(u.dex(u.cm**-2)))

Wavelength 0.2730793613528515 cm
hv/k 5.268713352690394 K
E_l 0.0 Hz J s
Q_rot 3.2054533168060106
J_ex 5.171572992228938 Hz K s
J_bg 0.8913217569486075 Hz K s

C18O (1–0):
  Column density = 9.742930688732239 1 / cm2
  Error           = 13.988689612987239 dex(1 / cm2)
